# Import GitHub repository

In [1]:
import os
import logging
import sys

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
if not os.path.isdir('./owr'):
  !git clone -b icarl https://github.com/gianluigilopardo/Open-World-Recognition.git
  !mv 'Open-World-Recognition' 'owr'

# Import packages

In [4]:
from owr import ResNet
from owr.dataset import Subset
from owr.icarl import classify
from owr.icarl import incremental_train
from owr.icarl import update_representation
from owr.icarl import construct_exemplar_set
from owr.icarl import reduce_exemplars
from owr.icarl import generate_new_exemplars
from owr.models import compute_loss
from owr.models import train_network
from owr import params
from owr import utils
from owr.utils import get_classes_names
from owr.utils import get_task_indexes
from owr.utils import splitter
from owr.utils import map_splits
from owr.utils import get_classes
from owr.utils import get_indexes
import owr.models

In [5]:
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random

In [6]:
print(params.SEED)
print(params.NUM_WORKERS)

42
2


In [7]:
print(params.NUM_EPOCHS)

70


# Define Datasets

In [8]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
from torchvision import datasets
trainDS = datasets.cifar.CIFAR100( 'data', train=True, download=True, transform=train_transformer)
testDS = datasets.cifar.CIFAR100( 'data', train=False, download=True, transform=test_transformer)


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [10]:
splits = splitter()

In [11]:
splits

[[91, 49, 74, 23, 27, 21, 24, 85, 87, 11],
 [55, 81, 28, 32, 25, 29, 95, 97, 12, 20],
 [62, 92, 34, 38, 31, 35, 93, 13, 26, 47],
 [70, 40, 44, 37, 41, 94, 14, 66, 69, 9],
 [76, 48, 86, 64, 16, 18, 15, 17, 96, 72],
 [88, 54, 75, 22, 33, 19, 30, 84, 5, 10],
 [63, 90, 43, 46, 42, 45, 83, 6, 39, 65],
 [77, 56, 82, 67, 7, 8, 80, 79, 78, 2],
 [60, 98, 50, 51, 89, 4, 3, 58, 99, 1],
 [53, 68, 71, 36, 61, 52, 0, 59, 57, 73]]

# Define Network

In [12]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [13]:
exemplars = [None]*params.NUM_CLASSES

test_indexes =  []
accs_train = []
accs_test = []

In [14]:
for task in range(0, params.NUM_TASKS*params.TASK_SIZE, params.TASK_SIZE):
    train_indexes = get_task_indexes(trainDS, task)
    test_indexes = test_indexes + get_task_indexes(testDS, task)

    train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
    test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

    train_loader = DataLoader( train_dataset, num_workers=2, batch_size=params.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader( test_dataset, num_workers=2, batch_size=params.BATCH_SIZE , shuffle=True )
    print(task)
    ICaRL, exemplars = incremental_train(trainDS, ICaRL, exemplars, task, train_transformer)
    col = []
    for i,x in enumerate( splits[ :int(task/10) + 1]) : 
        v = np.array(x)
        col = np.concatenate( (col,v), axis = None)
        col = col.astype(int)
    mean = None
    total = 0.0
    running_corrects = 0.0
    
    for img, lbl, _ in train_loader:
        img = img.float().to(params.DEVICE)
        preds, mean = classify(img, exemplars, ICaRL, task, trainDS, mean)
        preds = preds.to(params.DEVICE)
        labels = map_splits(lbl, col).to(params.DEVICE)
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'train accuracy = {accuracy}')
    accs_train.append(accuracy)

    total = 0.0
    running_corrects = 0.0
    tot_preds = []
    tot_lab = []
    for img, lbl, _ in test_loader:
        img = img.float().to(params.DEVICE)
        preds, _ = classify(img, exemplars, ICaRL, task, trainDS, mean)
        preds = preds.to(params.DEVICE)
        labels = map_splits(lbl, col).to(params.DEVICE)
        tot_preds = np.concatenate( ( tot_preds, preds.data.cpu().numpy() ) )
        tot_lab = np.concatenate( (tot_lab, labels.data.cpu().numpy()  ) )
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'test accuracy = {accuracy}')
    accs_test.append(accuracy)
    PATH = "./model" + str(task) + ".pr"
    torch.save(ICaRL, PATH)

0
Step: 0, Epoch: 0, Loss: 0.028072629123926163, Accuracy: 0.199
Step: 0, Epoch: 1, Loss: 0.024411840364336967, Accuracy: 0.4172
Step: 0, Epoch: 2, Loss: 0.017444167286157608, Accuracy: 0.5196
Step: 0, Epoch: 3, Loss: 0.014791912399232388, Accuracy: 0.5742
Step: 0, Epoch: 4, Loss: 0.030452057719230652, Accuracy: 0.6052
Step: 0, Epoch: 5, Loss: 0.015126950107514858, Accuracy: 0.6204
Step: 0, Epoch: 6, Loss: 0.0224700178951025, Accuracy: 0.6504
Step: 0, Epoch: 7, Loss: 0.01558984350413084, Accuracy: 0.6712
Step: 0, Epoch: 8, Loss: 0.01586560718715191, Accuracy: 0.6856
Step: 0, Epoch: 9, Loss: 0.03124232031404972, Accuracy: 0.7074
Step: 0, Epoch: 10, Loss: 0.022226139903068542, Accuracy: 0.718
Step: 0, Epoch: 11, Loss: 0.009839735925197601, Accuracy: 0.698
Step: 0, Epoch: 12, Loss: 0.014344747178256512, Accuracy: 0.743
Step: 0, Epoch: 13, Loss: 0.01451204251497984, Accuracy: 0.7484
Step: 0, Epoch: 14, Loss: 0.01679885946214199, Accuracy: 0.7594
Step: 0, Epoch: 15, Loss: 0.0134592764079570

/content/owr/icarl.py:180: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image).unsqueeze(0).float()


udpate representation:  389 reduce exemplars: 0 costruct exemplars: 32
task: 0 train accuracy = 0.9696
task: 0 test accuracy = 0.864
10
Step: 10, Epoch: 0, Loss: 0.0320826917886734, Accuracy: 0.2737142857142857
Step: 10, Epoch: 1, Loss: 0.027395054697990417, Accuracy: 0.3365714285714286
Step: 10, Epoch: 2, Loss: 0.026339838281273842, Accuracy: 0.3922857142857143
Step: 10, Epoch: 3, Loss: 0.02417885884642601, Accuracy: 0.43714285714285717
Step: 10, Epoch: 4, Loss: 0.025629103183746338, Accuracy: 0.47585714285714287
Step: 10, Epoch: 5, Loss: 0.022208230569958687, Accuracy: 0.5074285714285715
Step: 10, Epoch: 6, Loss: 0.024108005687594414, Accuracy: 0.535
Step: 10, Epoch: 7, Loss: 0.02260814607143402, Accuracy: 0.5491428571428572
Step: 10, Epoch: 8, Loss: 0.0230155810713768, Accuracy: 0.5687142857142857
Step: 10, Epoch: 9, Loss: 0.021833615377545357, Accuracy: 0.58
Step: 10, Epoch: 10, Loss: 0.021371478214859962, Accuracy: 0.6054285714285714
Step: 10, Epoch: 11, Loss: 0.020973607897758484

In [15]:
#Locura sugli exemplars